In [2]:
import pandas as pd
from IPython.core.display import display

In [15]:
df = pd.read_pickle('../data/coref_with_translations.pkl')
display(df)

,Doc_ID,Text_ID,Text,Sentence_lengths,Coref_Clusters,Text_trans,Sentence_lengths_trans,Alignments,Alignment_dict,Coref_clusters_trans
0,bc/cctv/00/cctv_0005,bc/cctv/00/cctv_0005-0,"[[--, basically, ,, it, was, unanimously, agre...","[14, 22, 29, 11, 5, 6, 12, 5]","[[[16, 16], [19, 23]], [[25, 27], [42, 44], [5...","[[-, i, princip, enades, det, enhälligt, av, d...","[12, 18, 23, 12, 6, 7, 13, 5]","[[[0, 0], [1, 1], [1, 2], [2, 1], [3, 3], [3, ...","{0: 0, 1: 2, 2: 1, 3: 4, 7: 5, 8: 6, 9: 7, 10:...","[[[15, 15], [18, 22]], [[23, 25], [36, 36], [4..."
1,bc/cctv/00/cctv_0005,bc/cctv/00/cctv_0005-1,"[[Hello, ,, dear, viewers, .], [Welcome, to, F...","[5, 5, 22, 37, 11, 13, 18, 15, 35, 19, 26, 2, ...","[[[7, 8], [233, 234]], [[10, 10], [172, 172]],...","[[Hej, kära, tittare, .], [Välkommen, till, Fo...","[4, 5, 20, 36, 9, 6, 15, 12, 26, 18, 21, 2, 13...","[[[0, 0], [2, 1], [3, 2], [4, 3]], [[0, 0], [1...","{0: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 7: 6, 8: ...","[[[6, 7], [197, 198]], [[9, 9], [142, 142]], [..."
2,bc/cctv/00/cctv_0005,bc/cctv/00/cctv_0005-2,"[[Well, ,, what, in, fact, was, the, content, ...","[12, 11, 70, 15, 18, 38, 22, 22, 2, 17, 2, 11,...","[[[9, 10], [25, 26], [103, 104], [116, 117], [...","[[Tja, ,, vad, var, faktiskt, innehållet, i, d...","[10, 12, 65, 12, 18, 31, 18, 20, 4, 14, 2, 10,...","[[[0, 0], [1, 1], [2, 2], [4, 4], [5, 3], [7, ...","{0: 0, 1: 1, 2: 2, 4: 4, 5: 3, 7: 5, 8: 6, 9: ...","[[[7, 8], [26, 26], [96, 96], [106, 107], [121..."
3,bc/cctv/00/cctv_0005,bc/cctv/00/cctv_0005-3,"[[We, usually, make, it, sound, like, e-govern...","[11, 21, 2, 26, 39, 6, 2, 32, 36, 2, 14, 27, 2...","[[[17, 17], [22, 22]], [[29, 30], [42, 43]], [...","[[Vi, brukar, låta, det, låta, som, e, -, förv...","[13, 17, 2, 29, 30, 6, 4, 28, 38, 4, 15, 27, 4...","[[[0, 0], [1, 1], [2, 2], [3, 3], [4, 4], [5, ...","{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 8, 7: ...","[[[20, 20], [24, 24]], [[28, 28], [40, 40]], [..."
4,bc/cctv/00/cctv_0005,bc/cctv/00/cctv_0005-4,"[[So, ,, how, exactly, did, citizens, react, ?...","[8, 11, 4, 8, 13, 8, 4, 6, 4, 4, 17, 3, 6, 8, ...","[[[23, 23], [33, 33], [45, 45], [74, 74]], [[2...","[[Så, ,, hur, reagerade, medborgarna, exakt, ?...","[7, 9, 4, 10, 14, 7, 2, 6, 3, 2, 15, 3, 6, 6, ...","[[[0, 0], [1, 1], [2, 2], [3, 3], [4, 3], [5, ...","{0: 0, 1: 1, 2: 2, 3: 3, 4: 3, 5: 4, 6: 5, 7: ...","[[[20, 20], [33, 33], [45, 45], [68, 68]], [[2..."
5,bc/cctv/00/cctv_0005,bc/cctv/00/cctv_0005-5,"[[Well, ,, ah, ,, Yang, Yang, ,, you, work, at...","[32, 4, 9, 17, 27, 19, 14, 52, 2, 15, 37, 41, ...","[[[4, 5], [7, 7], [83, 83], [94, 94], [333, 33...","[[Tja, ,, ah, ,, Yang, Yang, ,, du, arbetar, p...","[31, 4, 9, 15, 23, 17, 12, 46, 4, 14, 30, 38, ...","[[[0, 0], [1, 1], [2, 2], [3, 3], [4, 4], [5, ...","{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: ...","[[[4, 5], [7, 7], [78, 78], [87, 87], [295, 29..."
6,bc/cctv/00/cctv_0005,bc/cctv/00/cctv_0005-6,"[[Yes, ,, many, ,, viewers, not, in, Beijing, ...","[32, 12, 22, 8, 2, 66, 16, 2, 22, 42, 17, 2, 4...","[[[7, 7], [171, 172]], [[28, 29], [258, 261], ...","[[Ja, ,, många, ,, tittare, som, inte, är, i, ...","[34, 12, 19, 7, 4, 58, 15, 2, 19, 39, 17, 2, 3...","[[[0, 0], [1, 1], [2, 2], [3, 3], [4, 4], [5, ...","{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 6, 6: 8, 7: ...","[[[9, 9], [161, 161]], [[31, 31], [242, 245], ..."
7,bc/cctv/00/cctv_0005,bc/cctv/00/cctv_0005-7,"[[Uh-huh, .], [Well, ,, what, measures, did, t...","[2, 15, 23, 8, 14, 27, 28, 21, 21, 49, 11, 13,...","[[[7, 10], [30, 38], [50, 53], [72, 72], [149,...","[[Uh, -, va, .], [Tja, ,, vilka, åtgärder, vid...","[4, 12, 20, 8, 6, 19, 28, 16, 17, 45, 9, 10, 7...","[[[0, 2], [1, 3]], [[0, 0], [1, 1], [2, 2], [3...","{0: 2, 1: 3, 2: 4, 3: 5, 4: 6, 5: 7, 6: 8, 9: ...","[[[9, 9], [29, 34], [44, 46], [57, 57], [123, ..."
8,bc/cctv/00/cctv_0005,bc/cctv/00/cctv_0005-8,"[[Ah, ,, it, can, be, said, ,, ah, ,, Yang, Ya...","[50, 2, 4, 16, 22, 47, 32, 14, 9, 30, 43, 33, ...","[[[9, 10], [58, 58], [66, 66], [226, 226], [27...","[[Ah, ,

In [14]:
def insert_or_append(d, key, value):
    if key in d:
        d[key] = d.get(key) + '|' + value
    else:
        d[key] = value
    return d

coref_clusters = df['Coref_Clusters']
conll_coref_dicts = []
for clusters in coref_clusters:
    conll_coref_dict = {}
    conll_cluster_count = 0
    for cluster in clusters:
        for [span_start, span_end] in cluster:
            conll_value = ''
            if span_start == span_end:
                conll_value = '(' + str(conll_cluster_count) + ')'
                insert_or_append(conll_coref_dict, span_start, conll_value)
            else:
                conll_start_value = '(' + str(conll_cluster_count)
                conll_end_value = str(conll_cluster_count) + ')'
                insert_or_append(conll_coref_dict, span_start, conll_start_value)
                insert_or_append(conll_coref_dict, span_end, conll_end_value)
        conll_cluster_count += 1
    conll_coref_dicts.append(conll_coref_dict)
print(conll_coref_dicts)


[{16: '(0)', 19: '(0', 23: '0)', 25: '(1', 27: '1)', 42: '(1', 44: '1)', 57: '(1', 59: '1)', 82: '(2)', 83: '(2)'}, {7: '(0', 8: '0)', 233: '(0', 234: '0)', 10: '(1)', 172: '(1)', 27: '(2)', 59: '(2', 60: '2)|6)', 72: '(2', 73: '2)', 29: '(3', 30: '3)|4)', 108: '(3', 109: '3)', 112: '(3', 113: '3)', 18: '(4', 64: '(4', 67: '4)', 259: '(4', 260: '4)|5)', 411: '(4', 412: '4)', 516: '(4)', 35: '(5', 36: '5)', 256: '(5', 268: '(5)', 367: '(5', 368: '5)', 51: '(6', 383: '(6', 384: '6)', 166: '(7', 168: '7)', 182: '(7', 185: '7)', 224: '(7', 226: '7)', 247: '(7', 250: '7)', 170: '(8', 171: '8)', 187: '(8', 188: '8)', 228: '(8', 229: '8)', 190: '(9', 204: '9)', 272: '(9)', 281: '(9)', 292: '(9)', 314: '(9)', 325: '(9)', 334: '(9)', 432: '(9', 433: '9)', 216: '(10', 219: '10)|11)', 372: '(10', 373: '10)', 392: '(10', 393: '10)', 418: '(10', 419: '10)', 210: '(11', 350: '(11)', 352: '(11', 353: '11)', 360: '(11)', 362: '(11)', 375: '(11)', 387: '(11)', 408: '(11)', 238: '(12)', 435: '(12)', 493

In [23]:
# path = '../allennlp-model/data/train/train.swedish.conll'
path = 'test.conll'
texts = df['Text_trans']
full_ids = df['Text_ID']

with open(path, 'w') as file:
    for text, full_id, corefs in zip(texts, full_ids, conll_coref_dicts):
        [doc_id, text_id] = full_id.split('-')
        text_idx = 0
        file.write('#begin document (%s); part %s\n' %(doc_id, text_id.zfill(3)))
        for sentence in text:
           for idx, word in enumerate(sentence):
               coref = corefs.get(text_idx, '-')
               file.write(doc_id + '\t')
               file.write(text_id + '\t')
               file.write(str(idx) + '\t')
               file.write('{:20s}'.format(word))
               file.write('{:20s}'.format('XX'))
               file.write('-' + '\t' + '-' + '\t' + '-' + '\t' + '-' + '\t' + '-' + '\t')
               # file.write('{:40s}'.format(df.loc[i, 'Speaker']))
               file.write('-' + '\t')
               file.write(coref)
               file.write('\n')
               text_idx += 1

        file.write('\n#end document\n')